In [24]:
import tensorflow as tf

In [25]:
import os

In [26]:
data_dir = "/media/shyam/New Volume/cifar10/cifar-10-batches-bin"

In [27]:
'''crete a list of filenames, nothing special here'''
filenames = [os.path.join(data_dir,"data_batch_%d.bin"%i) for i in range(1,6)]

In [28]:
'''string_input_producer returns two things a QUEUE and a QUEUERUNNER
1. My personal opinion is not to specify num_epochs here, rather specify number of times for loop should run during training
2. if shuffle is used, seed must be specified
'''
filequeue = tf.train.string_input_producer(filenames,shuffle = True, seed = 4)

In [29]:
'''
1. Each execution of read reads a single line from the file. 
   The decode_csv/decode_raw op then parses the result into a list of tensors. 
   The record_defaults argument determines the type of the resulting tensors and sets the default value to use if a value is missing in the input string.

2. You must call tf.train.start_queue_runners to populate the queue before you call run or eval to execute the read. 
   Otherwise read will block while it waits for filenames from the queue.
'''

'\n1. Each execution of read reads a single line from the file. \n   The decode_csv/decode_raw op then parses the result into a list of tensors. \n   The record_defaults argument determines the type of the resulting tensors and sets the default value to use if a value is missing in the input string.\n\n2. You must call tf.train.start_queue_runners to populate the queue before you call run or eval to execute the read. \n   Otherwise read will block while it waits for filenames from the queue.\n'

In [30]:
'''Now we need a person(reader) to read from the queue'''
image_height = 32
image_width = 32
image_depth = 3
image_bytes = image_height * image_width * image_depth
label_bytes = 1
example_bytes = label_bytes + image_bytes 
reader = tf.FixedLengthRecordReader(record_bytes = example_bytes)
'''Enable the reader to read from the filequeue'''
key , value = reader.read(filequeue)
'''each byte contains one pixel so convert it in form of tf.uint8'''
record_bytes = tf.decode_raw(value, tf.uint8)

In [31]:
example_label = tf.cast(tf.slice(record_bytes,[0],[label_bytes]) , tf.int32)
example_image = tf.reshape(tf.slice(record_bytes, [label_bytes], [image_bytes]),
                           [image_depth, image_height, image_width])

In [32]:
sess = tf.InteractiveSession()
init_all = tf.initialize_all_variables()
init_local = tf.initialize_local_variables()
'''
1. it's important to initialize all local variables before starting the queue runners
   change the sequence of below lines and get error :)
2. QUEUERUNNERS are also called threads
   so generally we will see, 
   coord = tf.train.Coordinator()
   threads = tf.start_queue_runners(sess = sess,coord = coord)
   <do stuff>
   coord.request_stop()
   coord.join(threads)
   If we don't stop threads here, we will geet error while closing the session
'''
sess.run([init_local])
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess = sess, coord = coord)

In [33]:
labels = []
images = []
for i in range(1000):
    labels.append(sess.run(example_label))
    images.append(sess.run(example_image))

In [34]:
'''
In document 1 we had run a for loop to generate a list of images and labels from the CIFAR10 data
Tensorflow provides a mechansim to replace this for loop with a op node
Thesse op nodes can be either tf.train.shuffle_batch or tf.train.batch
Theses op nodes do two things simultaneously, first create a queue where the reader puts examples
second, create a batch of examples from this queue and return this batch in form of a list
'''

'\nIn document 1 we had run a for loop to generate a list of images and labels from the CIFAR10 data\nTensorflow provides a mechansim to replace this for loop with a op node\nThesse op nodes can be either tf.train.shuffle_batch or tf.train.batch\nTheses op nodes do two things simultaneously, first create a queue where the reader puts examples\nsecond, create a batch of examples from this queue and return this batch in form of a list\n'

In [35]:
 images_batch,labels_batch = tf.train.batch(
        [example_image,example_label],
        batch_size=10,
        num_threads=16,
        capacity=100 + 3 * 10,
        enqueue_many=False)

In [47]:
'''
Since tf.train.batch/shuffle_batch creates a queue we need to start queue runners/threads using tf.train.start_queue_runners
What will happen if we forget to start the queue runners?
The program will get blocked and keep waiting for the threads, so it will show as it is running but actually its doing nothing
'''
threads2 = tf.train.start_queue_runners(sess = sess, coord = coord)

In [ ]:
'''
We need to use tf.train.start_queue_runners only once after zero state layout definition is finished
'''

In [38]:
'''
try using and see what error you get:
imgs= sess.run([images_batch])
Error : Expected 2 elements in the list given 1
'''
imgs,labs = sess.run([images_batch,labels_batch])

In [53]:
coord.request_stop()
coord.join(threads)
coord.join(threads2)

In [54]:
sess.close()